![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials: Meta-Models

# What is a `Meta-System`?! ![Experimental feature](images/experimental.svg)

A `Meta-System` is an simpler approximation of a complex `System`.

When exploring a design space, detailed simulation are usually too expensive to be evaluated at each intermediate tested point. But in order to get a better overall model, an approximated solution would help evaluating the potential of all points. One way to achieve this is to compute the solution with an advanced model or software at few discrete points and to interpolate between those points in exploration phases.

## Learn to use it

### Create data for a meta-system

Start with a `System`

In [ ]:
import logging
import numpy as np

logging.getLogger().setLevel(logging.WARNING)

In [ ]:
from cosapp.base import System, Port

class XPort(Port):
    def setup(self):
        self.add_variable('x')

class MyExp(System):
    def setup(self):
        self.add_input(XPort, 'p_in')
        self.add_output(XPort, 'p_out')

    def compute(self):
        self.p_out.x = np.exp(self.p_in.x)

Create a Design of Experiments on this `System`, using an adequate `Driver` (for example, the `LinearDoE` driver)

In [ ]:
from cosapp.drivers import LinearDoE, RunOnce
from cosapp.recorders import DataFrameRecorder

exp = MyExp('mysin')
doe = exp.add_driver(LinearDoE('doe'))
doe.add_recorder(DataFrameRecorder(raw_output=True))
doe.add_child(RunOnce('run'))

doe.add_input_var({'p_in.x': {'lower': 0., 'upper': 3., 'count': 15},})

exp.run_drivers()

df = doe.recorder.export_data()
df.head()

### Create a meta-system

Define its structure (should be consistent with the original `System`!)

In [ ]:
from cosapp.systems import MetaSystem

class MetaExp(MetaSystem):
    def setup(self):
        self.add_input(XPort, 'p_in')
        self.add_output(XPort, 'p_out')

Instantiate the `MetaSystem` with the data of your DoE. Change the default meta-system if necessary.

In [ ]:
from cosapp.utils.surrogate_models import ResponseSurface

meta_exp = MetaExp('metaexp', doe.recorder.export_data(), default_model=ResponseSurface)

In [ ]:
meta_doe = meta_exp.add_driver(LinearDoE('doe'))
meta_doe.add_recorder(DataFrameRecorder(raw_output=True))
meta_doe.add_child(RunOnce('run'))

meta_doe.add_input_var({'p_in.x': {'lower': 0., 'upper': 3., 'count': 40}})

meta_exp.run_drivers()

### Use it!

For the moment, your `Meta-System` is not trained on your data.
So finally, run it to trigger the training.

In [ ]:
meta_exp.run_once()

In [ ]:
import plotly.graph_objs as go

doe_df = doe.recorder.export_data()
meta_df = meta_doe.recorder.export_data()

go.Figure(
    data=[
        go.Scatter(x=doe_df['p_in.x'], y=doe_df['p_out.x'], name="exp"),
        go.Scatter(x=meta_df['p_in.x'], y=meta_df['p_out.x'], name="meta_exp")
    ],
    layout=go.Layout(
        xaxis={'title': 'p_in.x'},
        yaxis={'title': 'p_out.x'}
    )
)

## Learn more

### Computational time

A `Meta-System` can achieve time savings by running a simplified version of `System`. The gain will strongly depends on the following characteristics:
- the original `System` complexity
- the inputs count
- the outputs count
- the model itself

On the previous example, the `Meta-System` is **slower** than the original `System`.

Never forget that a `Meta-System` introduces approximations versus the original `System`!

### Available models

**CoSApp** comes with various models: ResponseSurface, FloatKrigingSurrogate, AnisotropicGP, IsotropicGP, etc.

They are available in *cosapp.surrogates*. More details are coming soon!

In [ ]:
from cosapp.utils.surrogate_models import (
    ResponseSurface,
    FloatKrigingSurrogate,
    LinearNearestNeighbor,
    RBFNearestNeighbor,
    WeightedNearestNeighbor,
)

name2class = {
    'Kriging': FloatKrigingSurrogate, 
    'Surface': ResponseSurface, 
    'LinearNN': LinearNearestNeighbor, 
    'RBFNN': RBFNearestNeighbor, 
    'WeightedNN': WeightedNearestNeighbor
}

df = doe.recorder.export_data()

traces = [
    go.Scatter(
        x=df['p_in.x'], 
        y=df['p_out.x'], 
        name="exp",
    )
]

# Run model fitting for all available models
for name, model in name2class.items():
    meta_exp = MetaExp('meta_exp', df, default_model=model)

    meta_doe = meta_exp.add_driver(LinearDoE('doe'))
    meta_doe.add_recorder(DataFrameRecorder(raw_output=True))
    meta_doe.add_child(RunOnce('run'))
    meta_doe.add_input_var({'p_in.x': {'lower': 0., 'upper': 3., 'count': 40}})

    meta_exp.run_drivers()
    meta_df = meta_doe.recorder.export_data()
    
    traces.append(
        go.Scatter(
            x=meta_df['p_in.x'], 
            y=meta_df['p_out.x'], 
            name=name,
        )
    )

go.Figure(
    data=traces,
    layout=go.Layout(
        xaxis={'title': 'p_in.x'},
        yaxis={'title': 'p_out.x'},
        hovermode='x',
    )
)